In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

In [2]:
train = pd.read_csv("../../Data/Kaggle/StoreSales/processed_train.csv")
test = pd.read_csv("../../Data/Kaggle/StoreSales/processed_test.csv")

In [3]:
train.head()

,id,store_nbr,family,sales,onpromotion,typeholiday,dcoilwtico,city,state,typestores,cluster,day_of_week,day,month,year
0,0,1,0,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
1,1,1,1,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
2,2,1,2,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
3,3,1,3,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
4,4,1,4,0.0,0,3,93.14,18,12,3,13,2,1,1,2013


In [4]:
test.head()

,id,store_nbr,family,onpromotion,typeholiday,dcoilwtico,city,state,typestores,cluster,day_of_week,day,month,year
0,3000888,1,0,0,4,46.8,18,12,3,13,3,16,8,2017
1,3000889,1,1,0,4,46.8,18,12,3,13,3,16,8,2017
2,3000890,1,2,2,4,46.8,18,12,3,13,3,16,8,2017
3,3000891,1,3,20,4,46.8,18,12,3,13,3,16,8,2017
4,3000892,1,4,0,4,46.8,18,12,3,13,3,16,8,2017


In [5]:
features = train.drop(['id','sales'], axis=1)
target = train['sales']

In [6]:
features.shape

(3000888, 13)

In [7]:
target.shape

(3000888,)

In [8]:
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

In [9]:
scaled_features.shape

(3000888, 13)

In [10]:
sequence_length = 10

In [11]:
sequences = []

In [12]:
for i in range(len(scaled_features) - sequence_length):
    seq = scaled_features[i:i+sequence_length]
    sequences.append(seq)

In [13]:
sequences = np.array(sequences)
target = np.array(target)

In [14]:
sequences.shape

(3000878, 10, 13)

In [15]:
target.size

3000888

In [16]:
min_length = min(len(sequences), len(target))
sequences = sequences[:min_length]
target = target[:min_length]

In [17]:
model = Sequential()
model.add(LSTM(units=50, input_shape=(sequences.shape[1], sequences.shape[2])))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

In [18]:
model.fit(sequences, target, epochs=50, batch_size=10000)

Epoch 1/50
301/301 [==============================] - 25s 78ms/step - loss: 1332168.2500
Epoch 2/50
301/301 [==============================] - 25s 82ms/step - loss: 1320648.2500
Epoch 3/50
301/301 [==============================] - 23s 78ms/step - loss: 1310986.0000
Epoch 4/50
301/301 [==============================] - 23s 76ms/step - loss: 1302070.5000
Epoch 5/50
301/301 [==============================] - 23s 76ms/step - loss: 1293783.6250
Epoch 6/50
301/301 [==============================] - 23s 78ms/step - loss: 1286067.2500
Epoch 7/50
301/301 [==============================] - 23s 76ms/step - loss: 1278873.3750
Epoch 8/50
301/301 [==============================] - 23s 76ms/step - loss: 1272172.5000
Epoch 9/50
301/301 [==============================] - 23s 75ms/step - loss: 1265937.6250
Epoch 10/50
301/301 [==============================] - 23s 75ms/step - loss: 1260151.7500
Epoch 11/50
301/301 [==============================] - 22s 74ms/step - loss: 1254797.5000
Epoch 12/50
301/301

In [19]:
from joblib import dump, load

dump(model, 'joblib/M12.joblib')

['joblib/M12.joblib']

In [20]:
test_data = test.drop('id', axis=1)

In [21]:
test_data.head()

,store_nbr,family,onpromotion,typeholiday,dcoilwtico,city,state,typestores,cluster,day_of_week,day,month,year
0,1,0,0,4,46.8,18,12,3,13,3,16,8,2017
1,1,1,0,4,46.8,18,12,3,13,3,16,8,2017
2,1,2,2,4,46.8,18,12,3,13,3,16,8,2017
3,1,3,20,4,46.8,18,12,3,13,3,16,8,2017
4,1,4,0,4,46.8,18,12,3,13,3,16,8,2017


In [22]:
scaled_test_features = scaler.transform(test_data)

In [23]:
test_sequences = []

In [24]:
for i in range(len(scaled_test_features) - sequence_length):
    seq = scaled_test_features[i:i+sequence_length]
    test_sequences.append(seq)

In [25]:
test_sequences = np.array(test_sequences)

In [26]:
test_predictions = model.predict(test_sequences)

891/891 [==============================] - 1s 514us/step


In [27]:
test_predictions = np.maximum(test_predictions, 0)

In [28]:
test_predictions = np.concatenate([np.zeros(10),test_predictions.flatten()])

In [29]:
submission = pd.DataFrame({'id': test['id'], 'sales': test_predictions.flatten()})

In [30]:
submission.to_csv('../../Data/Kaggle/StoreSales/submission_lstm.csv', index=False)

In [33]:
import joblib

joblib.dump(scaler, 'joblib/scaler.joblib')

['joblib/scaler.joblib']